viernes 03 de mayo de 2019  
  
_Benjamín Hernández Cortés_ - _Juan Pablo Rojas Rojas_  
_Departamento de Ingeniería Informática (DIINF)_  
_Universidad de Santiago de Chile (USACH)_


## Laboratorio 2 - Fundamentos de Aprendizaje Profundo con Redes Neuronales
___

El presente código está orientado hacia la implementación de un perceptrón multicapa o red neuronal de múltiples capas (Multi-Layer Neural Network). En primera oportunidad, se utilizará el perceptrón para clasificar y emular compuertas lógicas AND, OR y XOR, tanto de 2 como de 4 entradas. Luego, se realizará una clasificación trabajando con el conjunto de datos [wine](https://archive.ics.uci.edu/ml/datasets/wine) provenientes de la [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/index.php) que contiene 13 atributos en total y que intentan describir clases de vinos provenientes de 3 cultivos distintos.


#### Importación de bibliotecas
---

Las bibliotecas a emplear son:
- **Numpy:** Herramienta de computación científica, que nos permitirá trabajar a través de vectores
- **Pandas:** Para la manipulación y lectura de datos
- **Matplotlib:** Para la visualización gráfica de diversos datos de interés
- **Itertools:** Como herramienta para iteración de objetos
- **Scikit-learn:** Para la obtención del dataset _wine_ y el uso de herramientas de evaluación de desempeño para los perceptrones multicapa.

In [56]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import sklearn as skl
from sklearn.datasets import load_wine

#### Definición de la clase TruthTableGenerator
---

La clase TruthTableGenerator permite generar tablas de verdad para 3 tipos de compuertas lógicas: AND, OR y XOR.

En cuanto a las funciones definidas, se tiene:

| **Función**  | **Descripción**  |
| ------------ | ------------ |
| `generate_table(n_inputs, logic_gate)`  |  Genera una tabla de verdad de n-variables (*n_inputs*), basada en una compuerta lógica determinada (*logic_gate*)|

In [57]:
class TruthTableGenerator():
    
    def generate_table(n_inputs, logic_gate):
        table = its.product([0,1], repeat = n_inputs)
        table = pd.DataFrame(table)
        results = []
        
        if logic_gate == 'AND':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                results.append( all(row) )
            
        elif logic_gate == 'OR':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                results.append( any(row) )
                    
        elif logic_gate == 'XOR':
            for i in range(n_inputs ** 2):
                row = table.loc[[i]].values[0]
                tmp_result = row[0] ^ row[1]
                for j in range(2, n_inputs):
                    tmp_result = tmp_result ^ row[j]
                results.append( tmp_result )
        
        else:
            return None
        
        table['result'] = results
        return table

#### Definición de la clase Layer
---
La clase Layer permite representar una de las capas que conforma al Perceptrón Multicapa. Requiere de 3 parámetros:

| <p style='text-align: left;'>**Parámetro**</p> | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| `number_of_inputs` |  <p style='text-align: justify;'>Valor númerico entero. Define la cantidad de entradas que recibe la capa, la cual debe ser mayor que 0. Por defecto, se define el número de entradas como 1.</p>|
| `number_of_neurons` | <p style='text-align: justify;'>Valor numérico entero. Define la cantidad de neuronas que posee la capa, la cual debe ser mayor que 0. Por defecto, se define el número de neuronas como 1.</p> |
| `activation_function` | <p style='text-align: justify;'>Cadena de caracteres. Define la función de activación que empleará la capa, la cual puede ser sigmoide (_sigmoid_) o ReLU (_relu_).</p> |

En cuanto a las funciones definidas, se tiene:

| <p style='text-align: left;'>**Función**</p>  | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| `calculate_output(input_vector)` |  <p style='text-align: justify;'>Realiza una predicción para un conjunto de datos (*input_vector*), empleando los valores actuales de los pesos (*weights*) asociados a la capa.</p>|


In [17]:
class Layer:
    def __init__(self,
                 number_of_inputs = 1,
                 number_of_neurons = 1,
                 activation_function = 'sigmoid'):
        self.number_of_neurons = number_of_neurons
        self.number_of_inputs = number_of_inputs
        self.activation_function = activation_function
        self.weights = np.random.rand(self.number_of_inputs + 1, self.number_of_neurons)
        self.input = None
        self.output = None
        
    def calculate_output(input_vector):
        self.input = input_vector
        sum_ = np.matmul(np.append(input_vector, 1), self.weights)
        if self.activation_function == 'sigmoid': self.output = sigmoid(sum_)
        elif self.activation_function == 'relu': self.output = relu(sum_)
        else: self.output = None
        return self.output
        
            

class MLP:
    def __init__(data = None,
                 number_of_inputs = 1,
                 number_of_hidden_layers = 0,
                 number_of_neurons_for_each_layer = None,
                 activation_functions_for_each_layer = None):
        
        self.data_nrows, self.data_ncols = data.shape
        self.number_of_hidden_layers = number_of_hidden_layers
        self.number_of_neurons_for_each_layer = number_of_neurons_for_each_layer
        self.activation_functions_for_each_layer = activation_functions_for_each_layer
        self.layers = []
        
        # Primero se crea una capa escondida (hidden layer) según la cantidad de entradas
        # definidas para el perceptrón. Si no se define un número de capas escondidas,
        # entonces solamente se creará la capa de salida (output layer), recreando a un
        # perceptrón simple.
        
        self.layers.append(
            Layer(number_of_inputs = number_of_inputs,
                  number_of_neurons = number_of_neurons_for_each_layer[0],
                  activation_function = activation_functions_for_each_layer[0])
        )
        for i in range(1, self.number_of_hidden_layers + 1):
            self.layers.append(
                Layer(number_of_inputs = number_of_neurons_for_each_layer[i-1],
                      number_of_neurons = number_of_neurons_for_each_layer[i],
                      activation_function = activation_functions_for_each_layer[i])
            )
    
    def training(self, number_of_iterations):
        errors_series = []
        for i in range(n_times):
            errors = 0
            for j in range(self.data_nrows):
                row = self.data.loc[[j]].values[0]
                predicted_value = self.predict( np.append(row[:-1], 1) )
                error = row[-1] - predicted_value
                errors += int(error != 0.0)
                self.weights = self.weights + self.learning_rate * error * np.append(row[:-1], 1)
            errors_series.append(errors)
        return np.array(errors_series)
    
    def predict(self, input_vector, target_vector):
        output_vector = None
        for layer in self.layers:
            output_vector = layer.calculate_output(input_vector)
            input_vector = output_vector
            print(output_vector)
        return output_vector
    
    def adjust_weights(target_vector):
        flag = True
        errors = []
        deltas_values = []
        
        # Calculo del error para la capa de salida
        error = np.subtract(target_vector, self.layers[-1].output)
        delta = np.array(error * sigmoid_derivative(self.layers[-1].output))
        errors.append(error)
        deltas_values.append(delta)
        
        for i in list(reversed(range(len(self.layers))))[1:]:
            error = np.dot(deltas_values[-1], self.layers[i+1].weights)
            delta = np.array(error * sigmoid_derivate(self.layers[i].output))
            deltas_values.append(delta)
        
        deltas_values.reverse()
        for i in range(len(self.layers)):
            adjustment = np.dot(self.layers[i].input, deltas_values[i])
            self.layers[i].weights = self.layers[i].weights += adjustment
            
        
        
        
        
                
                
                
            
        
        
        
        
        
        

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    return sigmoid(x) * (1 -  sigmoid(x))

def relu(x):
    return np.maximum(0, x)
                
def sklearn_to_df(sklearn_dataset):
    df = pd.DataFrame(sklearn_dataset.data, columns=sklearn_dataset.feature_names)
    df['target'] = pd.Series(sklearn_dataset.target)
    return df

    

a = sklearn_to_df(load_wine())

b = TruthTableGenerator.generate_table(n_inputs=4, logic_gate='XOR')
perceptron = MLP(number_of_inputs=4,
                 number_of_hidden_layers=1,
                 number_of_neurons_for_each_layer=[3,1],
                 activation_functions_for_each_layer=['sigmoid', 'sigmoid'])

In [41]:
a = np.array([1,2,3])
b = np.array([2,4,10])

np.flip(np.subtract(a, b))

c = "hola mundo"
print(c[::-1])

a * sigmoid_derivative(b)

a = np.array([
    [1,2],
    [3,4],
    [5,6]
])

for letter in reversed(c):
    print(letter)
    print(c)
    
np.dot(np.array([1, 2]), np.array([3, 4]).T)

odnum aloh
o
hola mundo
d
hola mundo
n
hola mundo
u
hola mundo
m
hola mundo
 
hola mundo
a
hola mundo
l
hola mundo
o
hola mundo
h
hola mundo


11

In [44]:
np.matmul([.1,.2,.3,.4], 2 * np.random.random((4, 3)) - 1)

array([0.39371039, 0.33970832, 0.11194925])

In [49]:
error = np.subtract(np.array([0,1,1]), np.array([.1,-.1,-.23]))
delta = np.array(error * sigmoid_derivative(np.array([1,2,3])))
print(delta)

[-0.01966119  0.11549294  0.05556729]


In [55]:
a = [1,2,3,4]
for layer in reversed(a[:-1]):
    print(layer)
    
list(reversed(range(len(a))))[1:]

3
2
1


[2, 1, 0]